In [1]:
import geopandas as gpd
import libpysal
import matplotlib.pyplot as plt
import momepy
import osmnx as ox
import pandas
import folium
from bokeh.io import output_notebook
from bokeh.plotting import show
from clustergram import Clustergram
from rtree import index   #TODO: check KDtree
import os
import fiona

# for interactive plot, comment if it's causing issues
%matplotlib notebook 

load the data of the buildings dataFrameA and textures dataFrameB

In [2]:
root_folder = "Michaels_Data"   # the root of the folder containing all the data(the main Michael's data folder)

# List all .gdb directories in the folder
gdb_files = []
for dirpath, dirnames, filenames in os.walk(root_folder):
    # Check if the folder contains a "commondata" folder
    if "commondata" in dirnames:
        commondata_folder = os.path.join(dirpath, "commondata")
        commondata_folder = os.path.normpath(commondata_folder)

        # List all .gdb files in the "commondata" folder
        for f in os.listdir(commondata_folder):
            if f.endswith('.gdb'):
                gdb_files.append(os.path.join(commondata_folder, f))

# Print the collected .gdb files
for gdb in gdb_files:
    print(gdb)
print("GDB Files:", gdb_files)

Michaels_Data\חלקות\commondata\myproject16.gdb
Michaels_Data\חלקות\commondata\scratch.gdb
Michaels_Data\מרקמים\commondata\myproject16.gdb
Michaels_Data\מרקמים\commondata\scratch.gdb
Michaels_Data\קונטור בניינים\commondata\jps_reka.gdb
Michaels_Data\קונטור בניינים\commondata\scratch.gdb
Michaels_Data\רגישות\commondata\myproject16.gdb
GDB Files: ['Michaels_Data\\חלקות\\commondata\\myproject16.gdb', 'Michaels_Data\\חלקות\\commondata\\scratch.gdb', 'Michaels_Data\\מרקמים\\commondata\\myproject16.gdb', 'Michaels_Data\\מרקמים\\commondata\\scratch.gdb', 'Michaels_Data\\קונטור בניינים\\commondata\\jps_reka.gdb', 'Michaels_Data\\קונטור בניינים\\commondata\\scratch.gdb', 'Michaels_Data\\רגישות\\commondata\\myproject16.gdb']


In [4]:
# Choose a specific .gdb file (first one in this case)
texture_file = gdb_files[2]  # Modify if you want to choose a different .gdb
building_file = gdb_files[4]  # Modify if you want to choose a different .gdb

# List the layers in the selected .gdb
layers_texture = fiona.listlayers(texture_file)
layers_building = fiona.listlayers(building_file)
print("texture file:", texture_file)
print("building file:", building_file)

# Choose a specific layer within the .gdb
texture_layer = layers_texture[0]  # Modify if you want to choose a different layer
building_layer = layers_building[0]  # Modify if you want to choose a different layer

# Load the specific layer
textures = gpd.read_file(texture_file, layer=texture_layer)
buildings = gpd.read_file(building_file, layer=building_layer)

texture file: Michaels_Data\מרקמים\commondata\myproject16.gdb
building file: Michaels_Data\קונטור בניינים\commondata\jps_reka.gdb


Check the buildings that are striclty contained in the textures

In [23]:
# Function to check which polygon in gdfB contains the polygon in gdfA
def find_containing_polygon(polygon, B,spatial_index):
    # Get possible matches using bounding box intersection
    possible_matches_index = list(spatial_index.intersection(polygon.bounds))
    possible_matches = B.iloc[possible_matches_index]
    
    # Check for actual containment within those possible matches
    for idx, b_polygon in possible_matches.iterrows():
        if polygon.within(b_polygon.geometry):
            return idx  # Return the index of the containing polygon in gdfB
    return None  # If no containing polygon is found


In [8]:
# Function to find the polygon in gdfB with the largest intersection area
def find_largest_area_intersection(polygon, B,spatial_index):
    # Get possible matches using bounding box intersection
    possible_matches_index = list(spatial_index.intersection(polygon.bounds))
    possible_matches = B.iloc[possible_matches_index]

    max_area = 0
    best_match_idx = None

    # Check for actual intersection and find the largest area of intersection
    for idx, b_polygon in possible_matches.iterrows():
        intersection = polygon.intersection(b_polygon.geometry)
        if not intersection.is_empty:
            intersection_area = intersection.area
            if intersection_area > max_area:
                max_area = intersection_area
                best_match_idx = idx

    return best_match_idx  # Return the index of the polygon in gdfB with the largest intersection
    

In [35]:
def find_intersection(gdfA, gdfB, strictly_contained=False):
    """
    NOTE : make sure that gdfA and gdfB are in the same crs to avoid potential errors
    :param gdfA: the buildings dataFrame
    :param gdfB: the textures dataFrame
    :return: a new building dataFrame with the containing_polygon_index column
    """
    
    gdfA = gdfA.to_crs(gdfB.crs)  
    
    # Create a spatial index for gdfB using R-tree
    spatial_index = index.Index()
    for idx, geometry in gdfB.iterrows():
        spatial_index.insert(idx, geometry.geometry.bounds)
    
    if strictly_contained:
        # Apply the function to find the containing polygon index for each polygon in gdfA
        gdfA['containing_polygon_index'] = gdfA.geometry.apply(lambda x: find_containing_polygon(x, gdfB,spatial_index))
    else:
        # Apply the function to find the index of the polygon in gdfB with the largest intersection
        gdfA['containing_polygon_index'] = gdfA.geometry.apply(lambda x: find_largest_area_intersection(x, gdfB,spatial_index))
        
    return gdfA

In [43]:
# plot the unfitted buildings
unfittedBuildings = find_intersection(buildings, textures, strictly_contained=True)
unfittedBuildings = unfittedBuildings[unfittedBuildings['containing_polygon_index'].isna()]


# # Ensure both GeoDataFrames are in the same CRS
# if textures.crs != buildings.crs:
#     buildings = buildings.to_crs(textures.crs)

# Create a figure and axis
fig, ax = plt.subplots()

# Plot the second GeoDataFrame on the same axis
textures.plot(ax=ax, color='blue')

# Plot the first GeoDataFrame
unfittedBuildings.plot(ax=ax, color='red')
   
plt.show()

     סוג הערות                קטגוריה סיווג_מרקמים_12042024 שכונה  \
39  None     4  מרחבי הפרויקט הישראלי       הפרויקט הישראלי  None   

    Shape_Length     Shape_Area  \
39   3089.833238  147973.277003   

                                             geometry  
39  MULTIPOLYGON (((3915872.013 3731380.045, 39158...  
     סוג הערות                קטגוריה סיווג_מרקמים_12042024 שכונה  \
39  None     4  מרחבי הפרויקט הישראלי       הפרויקט הישראלי  None   

    Shape_Length     Shape_Area  \
39   3089.833238  147973.277003   

                                             geometry  
39  MULTIPOLYGON (((3915872.013 3731380.045, 39158...  
       סוג הערות                קטגוריה סיווג_מרקמים_12042024 שכונה  \
39    None     4  מרחבי הפרויקט הישראלי       הפרויקט הישראלי  None   
2192  None  None                   None               קמפוסים  None   

      Shape_Length     Shape_Area  \
39     3089.833238  147973.277003   
2192   1265.002261   59271.140833   

                              

KeyboardInterrupt: 

# The following code is for plotting experiments (not relevant)

In [30]:
A = buildings
B = textures
A = A.to_crs(B.crs)

In [32]:
# plot polygon x with the polygon y from gdf2 
polygon_gdf1 = A.iloc[0].geometry  # Polygon from row x of gdf1
polygon_gdf2 = B.iloc[39].geometry  # Polygon from row y of gdf2
print(polygon_gdf1.centroid)
print(polygon_gdf2.centroid)

# Create a plot
fig, ax = plt.subplots()

# Plot both polygons on the same axes
B.iloc[[39]].plot(ax=ax, color='green', edgecolor='black', label='gdf2 Polygon')
A.iloc[[0]].plot(ax=ax, color='blue', edgecolor='black', label='gdf1 Polygon')


plt.title('Polygon from gdf1 (Row 0) and gdf2 (Row 39)')
plt.show()

POINT (3916086.3974861386 3730867.502599462)
POINT (3916061.3655935423 3731121.998584734)


<IPython.core.display.Javascript object>

In [34]:
# Ensure both GeoDataFrames are in the same CRS
if textures.crs != buildings.crs:
    buildings = buildings.to_crs(textures.crs)

# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 10))

# Plot textures on the first subplot
textures.plot(ax=ax1, color='lightgrey', edgecolor='black')
ax1.set_title('Textures')

# Plot buildings on the second subplot
buildings.plot(ax=ax2, color='red', edgecolor='black')
ax2.set_title('Buildings')

# Display the plot
plt.show()

<IPython.core.display.Javascript object>